# Reglas de asociación

## Sobre el paquete xgboost:

El paquete **XGBoost** es una biblioteca de software de aprendizaje automático que implementa el algoritmo de Gradient Boosting. Se puede encontrar la documentación oficial del paquete [aquí](https://cran.r-project.org/web/packages/xgboost/xgboost.pdf).

Algunas de las características y funcionalidades principales del paquete incluyen:

- **Algoritmo de Gradient Boosting**: Implementa el algoritmo de Gradient Boosting, que es ampliamente utilizado en problemas de regresión y clasificación.

- **Eficiencia y rendimiento**: xgboost está diseñado para ser altamente eficiente y puede manejar grandes conjuntos de datos con rapidez y eficacia.

- **Regularización integrada**: Incluye opciones para aplicar regularización para evitar el sobreajuste y mejorar la generalización del modelo.

Es importante tener en cuenta que el éxito del análisis con **xgboost** depende en gran medida de la calidad del preprocesamiento de datos y la configuración adecuada de los hiperparámetros del modelo.

## Preprocesamiento:

Este tipo de análisis requiere un preprocesamiento de datos adecuado para garantizar la calidad del modelo entrenado. Algunas de las tareas de preprocesamiento que se pueden realizar incluyen:

- **Limpieza de datos**: Eliminar datos faltantes, duplicados y cualquier otro ruido que pueda afectar la calidad de las reglas de asociación.

- **Transformación de datos**: Convertir variables categóricas en variables numéricas mediante técnicas como one-hot encoding o label encoding.

- **Escalamiento de características**: Normalizar o estandarizar las características para asegurar que todas estén en la misma escala.

Una vez completado el preprocesamiento, se pueden utilizar las funciones proporcionadas por el paquete **xgboost** para entrenar y evaluar el modelo en los datos preparados.


#### Librerias requeridas

In [9]:
library(tidyverse)
source("Preprocesamiento/utils.R")
source("Preprocesamiento/plots.R")

#### Lectura del dataset

In [10]:
data <- read.csv("../data/Statlog_weights.csv", header = TRUE, sep = ",")

#### Estructura de los datos

In [11]:
data %>% str()

'data.frame':	1000 obs. of  23 variables:
 $ Class                                                   : int  1 2 1 1 2 1 1 1 1 2 ...
 $ Status.of.existing.checking.account                     : chr  "< 0 DM" "0 - 200 DM" "no checking account" "< 0 DM" ...
 $ Duration.in.month                                       : int  6 48 12 42 24 36 24 36 12 30 ...
 $ Credit.history                                          : chr  "critical account" "existing credits paid" "critical account" "existing credits paid" ...
 $ Purpose                                                 : chr  "radio/television" "radio/television" "education" "furniture/equipment" ...
 $ Credit.amount                                           : int  1169 5951 2096 7882 4870 9055 2835 6948 3059 5234 ...
 $ Savings.account.bonds                                   : chr  "unknown/no savings" "< 100 DM" "< 100 DM" "< 100 DM" ...
 $ Present.employment.since                                : chr  ">= 7 years" "1 - 4 years" "4 - 7 year

### Analisis de los datos

In [12]:
data %>% head()

,Class,Status.of.existing.checking.account,Duration.in.month,Credit.history,Purpose,Credit.amount,Savings.account.bonds,Present.employment.since,Installment.rate.in.percentage.of.disposable.income,Other.debtors...guarantors,...,Other.installment.plans,Housing,Number.of.existing.credits.at.this.bank,Job,Number.of.people.being.liable.to.provide.maintenance.for,Telephone,Foreign.worker,Gender,Marital.Status,Weights
,<int>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,...,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,1,< 0 DM,6,critical account,radio/television,1169,unknown/no savings,>= 7 years,4,none,...,none,own,2,skilled employee,1,"yes, registered",yes,male,single,1
2,2,0 - 200 DM,48,existing credits paid,radio/television,5951,< 100 DM,1 - 4 years,2,none,...,none,own,1,skilled employee,1,none,yes,female,divorced/separated/married,1
3,1,no checking account,12,critical account,education,2096,< 100 DM,4 - 7 years,2,none,...,none,own,1,unskilled resident,2,none,yes,male,single,1
4,1,< 0 DM,42,existing credits paid,furniture/equipment,7882,< 100 DM,4 - 7 years,2,guarantor,...,none,for free,1,skilled employee,2,none,yes,male,single,1
5,2,< 0 DM,24,delay in paying,car (new),4870,< 100 DM,1 - 4 years,3,none,...,none,for free,2,skilled employee,2,none,yes,male,single,1
6,1,no checking account,36,existing credits paid,education,9055,unknown/no savings,1 - 4 years,2,none,...,none,for free,1,unskilled resident,2,"yes, registered",yes,male,single,1


#### Consulta de valores nulos

In [13]:
col_na_summary(data)

,NA_count
,<dbl>
Class,0
Status.of.existing.checking.account,0
Duration.in.month,0
Credit.history,0
Purpose,0
Credit.amount,0
Savings.account.bonds,0
Present.employment.since,0
Installment.rate.in.percentage.of.disposable.income,0


#### Label encoding

Label encoder porque con one hot encoder se generan muchas columnas y el resultado empieza a ser peor

In [14]:
data <- label_encoder(data)

Equivalencias para la columna 'Status.of.existing.checking.account':
       valor_original valor_codificado
1          0 - 200 DM                1
2              < 0 DM                2
3           >= 200 DM                3
4 no checking account                4

Equivalencias para la columna 'Credit.history':
         valor_original valor_codificado
1      all credits paid                1
2      critical account                2
3       delay in paying                3
4 existing credits paid                4
5   no credits/all paid                5

Equivalencias para la columna 'Purpose':
        valor_original valor_codificado
1             business                1
2            car (new)                2
3           car (used)                3
4  domestic appliances                4
5            education                5
6  furniture/equipment                6
7               others                7
8     radio/television                8
9              repairs                9

#### Modificación del formato

In [15]:
data <- format_adapter(data)

## Reglas de asociación

In [16]:
write.csv(data, "../data/Statlog_XGB.csv", row.names = FALSE)